### Задание 1.
#### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

#### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import json


In [2]:
KEYWORDS = ['python', 'numpy', 'java', 'tps', 'jpeg']


In [3]:
req = requests.get('https://habr.com/ru/all/')
    

In [4]:
soup = BeautifulSoup(req.text)


In [5]:
articles = soup.find_all('article', class_='post')
len(articles)


20

In [6]:
data = pd.DataFrame()


In [7]:
def analyze_article(article):
    post_id = article.parent.attrs.get('id')
    post_id = int(post_id.split('_')[-1])
    date = article.find('span', class_='post__time').text
    title_block = article.find('a', class_='post__title_link')
    title = title_block.text
    link = title_block.attrs.get('href')
    preview = article.find('div', class_='post__text').text
    article_soup = BeautifulSoup(requests.get(link).text)
    article_full = article_soup.find('div', class_='post__text').text
    kw_match = 0
    for word in KEYWORDS:
        if word in str(title.lower()):
            kw_match += 1
        elif word in str(preview.lower()):
            kw_match += 1
        elif word in str(article_full.lower()):
            kw_match += 1
    if kw_match > 0:
        row = pd.DataFrame([{'Дата': date, 'Заголовок': title, 'Ссылка': link, 'Текст статьи': article_full}])
    else:
        row = {}
    return row

In [8]:
for article in articles:
    data = pd.concat([data, pd.DataFrame(analyze_article(article))])
    
    
    time.sleep(0.2)
    

In [9]:
data

,Дата,Заголовок,Ссылка,Текст статьи
0,сегодня в 14:08,Пишем комикс-приключение на Kotlin,https://habr.com/ru/post/564818/,"Всем привет! Сегодня вас ждет легкая статья, к..."
0,сегодня в 14:03,React испортил веб-разработку,https://habr.com/ru/company/vdsina/blog/564788/,В начале июня я посетил конференцию разработчи...
0,сегодня в 13:11,Продукт без тестирования,https://habr.com/ru/post/564816/,"Всем привет, друзья. Немного детектива вам в л..."
0,сегодня в 13:02,Превращаем старый телефон на Android в резервн...,https://habr.com/ru/company/ruvds/blog/564166/,"\n\r\nСегодня я покажу вам, как запустить полн..."
0,сегодня в 13:00,Применение JDBC в процессе ETL,https://habr.com/ru/post/564654/,В предыдущей статье мы упоминали о Java + JDBC...
0,сегодня в 08:55,Дружимся с ESP,https://habr.com/ru/post/547330/,"Здравствуйте, коллеги и энтузиасты!Последние п..."
0,вчера в 19:50,Как сделать многоточечную WebRTC-конференцию (...,https://habr.com/ru/company/flashphoner/blog/5...,"Во время карантина, который то ослабевает, то ..."
0,вчера в 18:38,"Лёгкая, гибкая, производительная обёртка над W...",https://habr.com/ru/company/skillfactory/blog/...,В 2020 году автор оригинальной статьи решил на...


### Задание 2.
#### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:  
EMAIL = [xxx@x.ru, yyy@y.com]  

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>  

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.  

In [10]:
EMAIL = ['aaa@yandex.ru', 'angel@gmail.com', 'xxxx@mail.ru', 'user@list.ru', 'xyz@rambler.com']


In [11]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

data = json.dumps({'emailAddresses': EMAIL})
headers = {
           'Accept': 'application/json, text/plain, */*',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'en-RU,en;q=0.9,ru-RU;q=0.8,ru;q=0.7,en-US;q=0.6',
           'Connection': 'keep-alive',
           'Content-Length': '36',
           'Content-Type': 'application/json;charset=UTF-8',
           'Host': 'identityprotection.avast.com',
           'Origin': 'https://www.avast.com',
           'Referer': 'https://www.avast.com/',
           'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
           'sec-ch-ua-mobile': '?0',
           'Sec-Fetch-Dest': 'empty',
           'Sec-Fetch-Mode': 'cors',
           'Sec-Fetch-Site': 'same-site',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
           'Vaar-Header-App-Build-Version': '1.0.0',
           'Vaar-Header-App-Product': 'hackcheck-web-avast',
           'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
           'Vaar-Version': '0'
       }

In [12]:
req = requests.post(URL, data=data, headers=headers)
req


<Response [200]>

In [13]:
req = req.json()
req


{'breaches': {'30177': {'breachId': 30177,
   'site': 'clubcomodin.com',
   'recordsCount': 16409,
   'description': 'In November 2020, a collection of over 23,000 breached sites was leaked on several hacking forums and Telegram channels. These breached sites originated from Cit0Day, a now-defunct private subscription service marketed towards criminals. The leaked data primarily includes email addresses and passwords that Cit0Day offered for a daily or monthly subscription fee.',
   'publishDate': '2020-12-10T00:00:00Z',
   'statistics': {'usernames': 0, 'passwords': 16409, 'emails': 16409}},
  '18154': {'breachId': 18154,
   'site': 'reddoorz.com',
   'recordsCount': 5888579,
   'description': 'In September 2020, the South Asian hospitality brand RedDoorz was allegedly breached. The stolen data contains passwords, email addresses and additional personal information. This breach is being privately shared on the internet.',
   'publishDate': '2020-11-19T00:00:00Z',
   'statistics': {'us

In [14]:
data = pd.DataFrame(req['breaches']).transpose()
breaches = data[['publishDate', 'site', 'description']]
breaches


,publishDate,site,description
30177,2020-12-10T00:00:00Z,clubcomodin.com,"In November 2020, a collection of over 23,000 ..."
18154,2020-11-19T00:00:00Z,reddoorz.com,"In September 2020, the South Asian hospitality..."
32800,2020-12-10T00:00:00Z,winning365.com,"In November 2020, a collection of over 23,000 ..."
24949,2020-12-03T00:00:00Z,russianchicago.com,"In November 2020, a collection of over 23,000 ..."
18178,2020-12-03T00:00:00Z,shredderstoo.com,"In November 2020, a collection of over 23,000 ..."
...,...,...,...
18122,2020-11-05T00:00:00Z,vimeworld.ru,"In January 2018, the Russian Minecraft server ..."
18221,2020-12-10T00:00:00Z,gostika.ru,"In November 2020, a collection of over 23,000 ..."
34009,2020-12-17T00:00:00Z,espace-fribourg.ch,"In November 2020, a collection of over 23,000 ..."
18110,2021-01-14T00:00:00Z,speedtrader.com,"In February 2020, the online trading platform ..."


#### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get  

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>  

In [15]:
with open('vk_token.txt', 'r') as file_object:
    TOKEN = file_object.read().strip()
    file_object.close()
    

In [16]:
REQUEST = 'https://api.vk.com/method/wall.get'
GROUP = 'netology'


In [17]:
params = {
    'access_token': TOKEN,
    'v': 5.84,
    'domain': GROUP,
    'count': 50
}


In [18]:
res = requests.get(REQUEST, params)
res


<Response [200]>

In [19]:
posts = pd.DataFrame(res.json()['response']['items'])[['date', 'text']]
posts


,date,text
0,1624695309,Интернет-маркетинг в рунете: полная история ра...
1,1624691700,В фильмах типа «Терминатора» или «Из машины» с...
2,1624629900,"Выбирайте, герой какой вселенной успеет закрыт..."
3,1624610100,Собрали подборку бесплатных курсов и интенсиво...
4,1624547940,"Согласно исследованию журнала Inc., общение с ..."
5,1624523100,И снова рассказываем о наших выпускниках 🔥 \n ...
6,1624461540,Отпуск — это конечно хорошо. Но никто не расск...
7,1624435980,🧸 28 июня стартует бесплатный курс «Soft Skill...
8,1624376880,"Не важно, о чём вы договариваетесь — о слиянии..."
9,1624364460,В эти выходные Нетология должна была принять у...


In [ ]:
TOKEN = ''